In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
from flask import Flask, request, jsonify

In [ ]:
app = Flask(__name__)

In [3]:
def read_data():
    data = pd.read_csv('./data/imdb_top_1000.csv')
    df = pd.DataFrame(data)
    return df

In [4]:
df = read_data()
df = df.drop(columns=['Certificate', 'Gross', 'Meta_score'])

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Overview'])
tfidf_matrix.shape
sim_vec = linear_kernel(tfidf_matrix , tfidf_matrix)
indexes = pd.Series(df.index , index = df['Series_Title'])

df['meta_soup'] = df['Genre'] +' '+ df['Director'] +' ' + df['Star1'] +' '+df['Star2'] +' '+ df['Star3'] +' ' + df['Star4']
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['meta_soup'])
sim_vec = cosine_similarity(count_matrix , count_matrix)
indexes = pd.Series(df.index , index = df['Series_Title'])

In [5]:
def recommend_movies(x , sim_vec):
    id_ = indexes[x]
    sim_movies = list(enumerate(sim_vec[id_]))
    sim_movies = sorted(sim_movies , key = lambda x : x[1] , reverse=True)
    sim_movies = sim_movies[1:11]
    R_movies = []
    for i in sim_movies:
        R_movies.append(i[0])
    return df[['Series_Title','Released_Year','IMDB_Rating','Genre','Director','Star1','Star2','Star3','Star4']].iloc[R_movies]


In [6]:
recommend_movies('The Dark Knight' , sim_vec)

,Series_Title,Released_Year,IMDB_Rating,Genre,Director,Star1,Star2,Star3,Star4
155,Batman Begins,2005,8.2,"Action, Adventure",Christopher Nolan,Christian Bale,Michael Caine,Ken Watanabe,Liam Neeson
36,The Prestige,2006,8.5,"Drama, Mystery, Sci-Fi",Christopher Nolan,Christian Bale,Hugh Jackman,Scarlett Johansson,Michael Caine
63,The Dark Knight Rises,2012,8.4,"Action, Adventure",Christopher Nolan,Christian Bale,Tom Hardy,Anne Hathaway,Gary Oldman
774,3:10 to Yuma,2007,7.7,"Action, Crime, Drama",James Mangold,Russell Crowe,Christian Bale,Ben Foster,Logan Lerman
953,American Psycho,2000,7.6,"Comedy, Crime, Drama",Mary Harron,Christian Bale,Justin Theroux,Josh Lucas,Bill Sage
217,Ford v Ferrari,2019,8.1,"Action, Biography, Drama",James Mangold,Matt Damon,Christian Bale,Jon Bernthal,Caitriona Balfe
241,Kill Bill: Vol. 1,2003,8.1,"Action, Crime, Drama",Quentin Tarantino,Uma Thurman,David Carradine,Daryl Hannah,Michael Madsen
573,Dunkirk,2017,7.8,"Action, Drama, History",Christopher Nolan,Fionn Whitehead,Barry Keoghan,Mark Rylance,Tom Hardy
782,Man on Fire,2004,7.7,"Action, Crime, Drama",Tony Scott,Denzel Washington,Christopher Walken,Dakota Fanning,Radha Mitchell
832,Empire of the Sun,1987,7.7,"Action, Drama, History",Steven Spielberg,Christian Bale,John Malkovich,Miranda Richardson,Nigel Havers
